# MNIST数字認識のシンプルなコード

## 必要なパッケージをインポート

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.optim as optim
import time
import matplotlib.pyplot as plt

## グローバル定数の設定

In [ ]:
batch_size = 200                # ミニバッチサイズ
sgd_lr = 0.1 # SGDの学習率

## データローダの準備 (MNISTデータのダウンロードも含む)

In [ ]:
root = '.' # mnistデータの置き場所，このフォルダに置くことを意味する
download = True
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = datasets.MNIST(root=root, train=True, transform=trans, download=download)
test_set = datasets.MNIST(root=root, train=False, transform=trans)
# ローダの準備
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

## データの中身を表示してみる

In [ ]:
data_iter = iter(train_loader)

In [ ]:
images, labels = next(data_iter)
npimg = images[0].numpy()
npimg = npimg.reshape((28, 28))
plt.imshow(npimg, cmap='gray')
print('Label:', labels[0].item())

## モデルの定義


In [ ]:
class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__() 
        self.l1 = nn.Linear(784, 32) # 28 x 28 = 784 次元の入力
        self.l2 = nn.Linear(32, 16)
        self.l3 = nn.Linear(16, 10)

    def forward(self, x):
        x = torch.sigmoid(self.l1(x))
        x = torch.sigmoid(self.l2(x))
        x = self.l3(x)
        return F.log_softmax(x, dim=1)

## 訓練ループ

In [ ]:
model = Net() # モデルのインスタンス生成
optimizer = optim.SGD(model.parameters(), lr=sgd_lr)
running_loss = 0.0
i = 0
for loop in range(3): # 3エポックの訓練
    for (input, target) in train_loader:
        i = i + 1
        input = input.view(-1, 28*28) # テンソルのサイズを整える
        optimizer.zero_grad()    # optimizerの初期化
        output = model(input)     # 推論計算
        loss = F.nll_loss(output, target) # 損失関数の定義
        loss.backward()             # バックプロパゲーション(後ろ向き計算)
        optimizer.step()            # パラメータ更新
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%5d] loss: %.3f' %
                  (i + 1, running_loss / 100))
            running_loss = 0.0

## 精度の評価

In [ ]:
correct =  0 # 正解数
count = 0 # 試行数
with torch.no_grad():
    for (input, target) in test_loader:
        input = input.view(-1, 28*28)
        output = model(input)     # 推論計算
        pred = output.argmax(dim=1)
        correct += pred.eq(target.data).sum()
        count += batch_size
print ('accuracy = ', float(correct)/float(count)) # 正解率の表示

## （おまけ）テスト認識を実施してみる

入力画像に対して正解出力を出せているか確認する。

In [ ]:
test_iter = iter(test_loader)

In [ ]:
input, labels = next(test_iter)
input = input.view(-1, 28*28)
output = model(input)     # 推論計算
pred = output.argmax(dim=1)
plt.imshow(input[0].numpy().reshape(28, 28), cmap='gray')
print('True      Label:', labels[0].item())
print('Estimated Label:', pred[0].item())

---------------------


## 演習問題 5-0

上記のコードを実行しつつ、理解せよ。この問題についてはレポートによる報告は必要ない。

## 演習問題 5-1 
上のコードでは、活性化関数としてsigmoid関数を利用している。ReLU (torch.relu)に替えて実行し、そのときの訓練損失や正解率を報告し、考察せよ。（sigmoid関数やReLU関数の実際の関数形状は第２回のスライドにあります）

## 演習問題 5-2
以下の演習問題では、ReLU関数を利用したコードを利用せよ。
上記のコードの学習プロセスにおいては3エポック(訓練データを一通り全部使い切るのが１エポック) の学習を行なっている。1エポック、５エポックのときはどのような正解率となるか報告し、考察せよ。


## 演習問題 5-3
層数, 層の次元, 学習率など何を変更してもよいので、正解率の向上を目指せ。どこをどのように工夫したかと，そのときの結果を報告せよ。また、変更に応じて得られた結果の傾向を考察せよ。図や表を用いても良い。

-------------------
